## 기본 예시: 프롬프트 + 모델 + 출력 파서

가장 기본적이고 일반적인 사용 사례는 prompt 템플릿과 모델을 함께 연결하는 것입니다. 이것이 어떻게 작동하는지 보기 위해, 각 나라별 수도를 물어보는 Chain을 생성해 보겠습니다.


In [1]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

True

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH01-Basic")

LangSmith 추적을 시작합니다.
[프로젝트명]
CH01-Basic


## 프롬프트 템플릿의 활용

`PromptTemplate`

- 사용자의 입력 변수를 사용하여 완전한 프롬프트 문자열을 만드는 데 사용되는 템플릿입니다
- 사용법
  - `template`: 템플릿 문자열입니다. 이 문자열 내에서 중괄호 `{}`는 변수를 나타냅니다.
  - `input_variables`: 중괄호 안에 들어갈 변수의 이름을 리스트로 정의합니다.

`input_variables`

- input_variables는 PromptTemplate에서 사용되는 변수의 이름을 정의하는 리스트입니다.

In [3]:
from langchain_teddynote.messages import stream_response  # 스트리밍 출력
from langchain_core.prompts import PromptTemplate

`from_template()` 메소드를 사용하여 PromptTemplate 객체 생성


In [4]:
# template 정의
template = "{country}의 수도는 어디인가요?"

# from_template 메소드를 이용하여 PromptTemplate 객체 생성
prompt_template = PromptTemplate.from_template(template)
prompt_template

PromptTemplate(input_variables=['country'], template='{country}의 수도는 어디인가요?')

In [5]:
# prompt 생성
prompt = prompt_template.format(country="대한민국")
prompt

'대한민국의 수도는 어디인가요?'

In [6]:
# prompt 생성
prompt = prompt_template.format(country="미국")
prompt

'미국의 수도는 어디인가요?'

In [8]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(
    model="gpt-3.5-turbo",
    max_tokens=2048,
    temperature=0.1,
)

## Chain 생성

### LCEL(LangChain Expression Language)

![lcel.png](./images/lcel.png)

여기서 우리는 LCEL을 사용하여 다양한 구성 요소를 단일 체인으로 결합합니다

```
chain = prompt | model | output_parser
```

`|` 기호는 [unix 파이프 연산자](<https://en.wikipedia.org/wiki/Pipeline_(Unix)>)와 유사하며, 서로 다른 구성 요소를 연결하고 한 구성 요소의 출력을 다음 구성 요소의 입력으로 전달합니다.

이 체인에서 사용자 입력은 프롬프트 템플릿으로 전달되고, 그런 다음 프롬프트 템플릿 출력은 모델로 전달됩니다. 각 구성 요소를 개별적으로 살펴보면 무슨 일이 일어나고 있는지 이해할 수 있습니다.


In [17]:
# prompt 를 PromptTemplate 객체로 생성합니다.
prompt = PromptTemplate.from_template("{topic} 에 대해 {how} 설명해주세요.")

# model = ChatOpenAI()
model = ChatOpenAI(
    model="gpt-3.5-turbo",
    max_tokens=2048,
    temperature=0.1,
)

chain = prompt | model

In [18]:
chain

PromptTemplate(input_variables=['how', 'topic'], template='{topic} 에 대해 {how} 설명해주세요.')
| ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x0000027BE72A8790>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x0000027BE72ABB50>, temperature=0.1, openai_api_key=SecretStr('**********'), openai_proxy='', max_tokens=2048)

### invoke() 호출

- python 딕셔너리 형태로 입력값을 전달합니다.(키: 값)
- invoke() 함수 호출 시, 입력값을 전달합니다.

In [22]:
# input 딕셔너리에 주제를 '인공지능 모델의 학습 원리'으로 설정합니다.
input = {"topic": "인공지능 모델의 학습 원리", "how": "일본어로"}

In [23]:
# prompt 객체와 model 객체를 파이프(|) 연산자로 연결하고 invoke 메서드를 사용하여 input을 전달합니다.
# 이를 통해 AI 모델이 생성한 메시지를 반환합니다.
chain.invoke(input)

AIMessage(content='人工知能モデルの学習原理は、データを入力し、そのデータからパターンや規則性を学習することです。この学習過程では、モデルは入力データから特徴を抽出し、それらの特徴を使って問題を解決する方法を習得します。さらに、モデルはフィードバックを受け取り、誤差を最小化するように自身を調整していきます。このようにして、人工知能モデルはデータから学習し、問題を解決する能力を向上させていきます。', response_metadata={'token_usage': {'completion_tokens': 194, 'prompt_tokens': 34, 'total_tokens': 228}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-8a1ede24-d8a5-4072-b0a7-9a2bae75cf4b-0', usage_metadata={'input_tokens': 34, 'output_tokens': 194, 'total_tokens': 228})

아래는 스트리밍을 출력하는 예시 입니다.

In [16]:
# 스트리밍 출력을 위한 요청
answer = chain.stream(input)
# 스트리밍 출력
stream_response(answer)

인공지능 모델의 학습 원리는 데이터를 입력으로 받아서 패턴을 학습하는 과정입니다. 모델은 입력된 데이터를 분석하여 패턴을 찾아내고, 이를 기반으로 예측이나 분류를 수행합니다. 이 과정은 주어진 데이터와 정답 레이블을 이용하여 모델을 훈련시키는 것으로 이루어집니다. 모델은 훈련 데이터를 반복적으로 학습하면서 오차를 최소화하고 최적의 가중치와 편향을 찾아내는 과정을 거치게 됩니다. 이렇게 학습된 모델은 새로운 데이터를 입력받아 예측을 수행할 수 있습니다.

### 출력파서(Output Parser)


In [25]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

Chain 에 출력파서를 추가합니다.

In [26]:
# 프롬프트, 모델, 출력 파서를 연결하여 처리 체인을 구성합니다.
chain = prompt | model | output_parser

In [27]:
# chain 객체의 invoke 메서드를 사용하여 input을 전달합니다.
input = {"topic": "인공지능 모델의 학습 원리", "how": "간단하게"}
chain.invoke(input)

'인공지능 모델의 학습 원리는 데이터를 입력으로 받아들이고, 이 데이터를 분석하여 패턴이나 규칙을 학습하는 과정을 말합니다. 이러한 학습 과정은 주어진 데이터와 원하는 결과를 매칭시키는 방식으로 이루어지며, 모델은 이러한 데이터와 결과 사이의 관계를 학습하여 새로운 데이터에 대해 예측이나 분류를 수행할 수 있게 됩니다. 이러한 학습 원리를 통해 인공지능 모델은 점차적으로 더 나은 성능을 발휘하게 되며, 다양한 분야에서 활용될 수 있습니다.'

In [28]:
# 스트리밍 출력을 위한 요청
answer = chain.stream(input)
# 스트리밍 출력
stream_response(answer)

인공지능 모델의 학습 원리는 데이터를 입력으로 받아들이고, 이 데이터를 분석하여 패턴을 학습하는 과정을 말합니다. 이 과정은 크게 입력층, 은닉층, 출력층으로 구성된 인공신경망을 사용하여 이루어집니다. 

먼저, 입력층에는 데이터가 입력되고, 은닉층에서는 입력된 데이터를 분석하고 패턴을 학습합니다. 이후, 출력층에서는 학습된 패턴을 바탕으로 결과를 출력합니다. 

학습 과정에서는 모델이 예측한 결과와 실제 결과 간의 차이를 계산하여 이 차이를 최소화하는 방향으로 모델을 업데이트하며 학습이 진행됩니다. 이러한 과정을 반복하여 모델이 데이터의 패턴을 더욱 정확하게 학습하게 됩니다.

### 템플릿을 변경하여 적용

- 아래의 프롬프트 내용을 얼마든지 **변경** 하여 테스트 해볼 수 있습니다.
- `model_name` 역시 변경하여 테스트가 가능합니다.

In [29]:
template = """
당신은 영어를 가르치는 10년차 영어 선생님입니다. 주어진 상황에 맞는 영어 회화를 작성해 주세요.
양식은 [FORMAT]을 참고하여 작성해 주세요.

#상황:
{question}

#FORMAT:
- 영어 회화:
- 한글 해석:
"""

# 프롬프트 템플릿을 이용하여 프롬프트를 생성합니다.
prompt = PromptTemplate.from_template(template)

# ChatOpenAI 챗모델을 초기화합니다.
model = ChatOpenAI(model_name="gpt-4o-mini")

# 문자열 출력 파서를 초기화합니다.
output_parser = StrOutputParser()

In [30]:
# 체인을 구성합니다.
chain = prompt | model | output_parser

In [31]:
# 완성된 Chain을 실행하여 답변을 얻습니다.
print(chain.invoke({"question": "저는 식당에 가서 음식을 주문하고 싶어요"}))

- 영어 회화:
   A: Hi there! Welcome to our restaurant. Can I take your order?
   B: Yes, I’d like to have the grilled chicken salad, please.
   A: Great choice! Would you like anything to drink with that?
   B: I’ll have a glass of iced tea, thank you.
   A: Perfect! Your order will be ready shortly.

- 한글 해석:
   A: 안녕하세요! 저희 식당에 오신 것을 환영합니다. 주문하시겠어요?
   B: 네, 구운 치킨 샐러드를 주문하고 싶어요.
   A: 좋은 선택이세요! 음료는 무엇을 드릴까요?
   B: 아이스 티 한 잔 주세요, 감사합니다.
   A: 완벽해요! 주문하신 음식은 곧 준비될 거예요.


In [32]:
# 완성된 Chain을 실행하여 답변을 얻습니다.
# 스트리밍 출력을 위한 요청
answer = chain.stream({"question": "저는 식당에 가서 음식을 주문하고 싶어요"})
# 스트리밍 출력
stream_response(answer)

- 영어 회화:  
  **Waiter:** Good evening! Welcome to our restaurant. How many people are in your party?  
  **You:** Good evening! It's just me.  
  **Waiter:** Great! Here’s the menu. Can I start you off with something to drink?  
  **You:** Yes, I’d like a glass of water, please.  
  **Waiter:** Sure! Are you ready to order your meal?  
  **You:** Yes, I’d like the grilled chicken with vegetables, please.  
  **Waiter:** Excellent choice! Would you like any sides with that?  
  **You:** Yes, could I have a side of mashed potatoes?  
  **Waiter:** Of course! Is there anything else I can get for you?  
  **You:** No, that’s all for now. Thank you!  
  **Waiter:** You’re welcome! I’ll be right back with your drink.

- 한글 해석:  
  **웨이터:** 좋은 저녁입니다! 저희 식당에 오신 것을 환영합니다. 몇 분이세요?  
  **당신:** 좋은 저녁입니다! 저 혼자입니다.  
  **웨이터:** 좋습니다! 여기 메뉴입니다. 음료수로 무엇을 드릴까요?  
  **당신:** 네, 물 한 잔 주세요.  
  **웨이터:** 알겠습니다! 식사 주문하실 준비가 되셨나요?  
  **당신:** 네, 구운 치킨과 야채를 주문하고 싶습니다.  
  **웨이터:** 좋은 선택입니다! 그와 함께 사이드 메뉴가 필요하신가

In [ ]:
# 이번에는 question 을 '미국에서 피자 주문'으로 설정하여 실행합니다.
# 스트리밍 출력을 위한 요청
answer = chain.stream({"question": "미국에서 피자 주문"})
# 스트리밍 출력
stream_response(answer)